In [ ]:
import os
import warnings
import json
import time
import numpy as np
import random
import torch
import argparse
import src
from pathlib import Path
from sklearn.model_selection import train_test_split
import segmentation_models_pytorch as smp



In [5]:

import os
import warnings
import json
import time
import numpy as np
from tqdm import tqdm
import random
import torch
import rasterio
import argparse
import src
from pathlib import Path
from sklearn.model_selection import train_test_split
import segmentation_models_pytorch as smp

warnings.filterwarnings("ignore")

In [6]:
CLASSES = [0, 1, 2, 3]
LR = 1e-4
IMG_SIZE = 512
BATCH_SIZE = 12
NUM_EPOCHS = 150
# CLS_WEIGHTS = [0.0048, 0.0302, 0.5828, 0.3822]
# WEIGHTS_DIR = "weights_unet"


CLASSWEIGHT

In [7]:
TARGET_DIR = "/home/chia/bdd_multimodal-main/BDD_dataset/train/target"


def compute_class_weights(target_dir, classes):
    pixel_counts = {cls: 0 for cls in classes}  # Initialize pixel counts for each class
    total_pixels = 0

    # Iterate over all ground truth masks
    for mask_path in tqdm(Path(target_dir).rglob("*.tif"), desc="Counting Pixels"):
        with rasterio.open(mask_path) as src:
            mask = src.read(1)  # Read the mask (1 channel)

            # Count pixels for each class
            for cls in classes:
                pixel_counts[cls] += np.sum(mask == cls)
            
            total_pixels += mask.size  # Total pixels in the mask

    # Compute weights
    class_weights = {
        cls: total_pixels / pixel_counts[cls] if pixel_counts[cls] > 0 else 0
        for cls in classes
    }

    # Normalize weights to sum up to 1
    total_weight = sum(class_weights.values())
    normalized_weights = {
        cls: weight / total_weight for cls, weight in class_weights.items()
    }

    return normalized_weights, class_weights, pixel_counts

# Compute class weights
normalized_weights, class_weights, pixel_counts = compute_class_weights(TARGET_DIR, CLASSES)

# Output results
print("Pixel Counts per Class:", pixel_counts)
print("Original Class Weights:", class_weights)
print("Normalized Class Weights (Sum to 1):", normalized_weights)

Counting Pixels: 549it [00:01, 422.21it/s]

Pixel Counts per Class: {0: np.int64(114027880), 1: np.int64(7918203), 2: np.int64(7411312), 3: np.int64(14559661)}
Original Class Weights: {0: np.float64(1.2621216495474616), 1: np.float64(18.17546935838851), 2: np.float64(19.41856664515001), 3: np.float64(9.884643330637987)}
Normalized Class Weights (Sum to 1): {0: np.float64(0.025894561108442234), 1: np.float64(0.37290050617875264), 2: np.float64(0.3984047502960499), 3: np.float64(0.20280018241675532)}
